# Face Transformation from Human to Anime
This notebook demonstrates how to transform a human face to an anime face using StyleGAN2.

Contact: alienncheng@gmail.com

## Data Preparation
First of all, we need a dataset to train the model.
I downloaded the [danbooru2019-portrait](https://www.gwern.net/Crops#danbooru2019-portraits) dataset and manually pick 1000 images from it.

Then we run the dataset tool provided by StyleGAN2 to create TensorFlow records.

#### Note
1. As [Doron Adler's work](https://twitter.com/Buntworthy/status/1297976798236598274) shows, 317 images should be enough.
2. Beware of mode issues. The model never learn something not (or seldomly) appearing in the dataset. Due to the extreme gender ratio of the anime portraits, the model would eventually learn how to generate anime faces excluding male faces.

In [ ]:
!python dataset_tool.py create_from_images datasets/custom_512 datasets_img/

## Train a StyleGAN2 network to generate anime portrait
We need a network to generate anime portrait.

I suggest transfer learning from a well-trained network so that we can not only save the time but also somehow keep the latent space of the source network.

Here I fine-tuned the model [stylegan2-ffhq-512-config-f](https://twitter.com/AydaoAI/status/1269689136019116032?s=20) with the dataset above.

In [ ]:
!python run_training.py --result-dir=D://Anime/results/ --data-dir=datasets/ --dataset=custom_512 --config=config-f --total-kimg=200

## Align a human face image
An aligned face image is more understandable for model. Furthermore, we need the image cropped in a suitable manner.

In [ ]:
!python align_images.py images/raw images/aligned

So far we have the essential materials prepared, a human face image, a ffhq model and a anime face model, both of which share the latent space (or similar at least).

Then what we need to do next is to tranform the human face to an anime face.

Here we have some choices to get our works done:
1. Extract the latent code of the given human face image, then simply insert it to the anime model.
2. Blend the human model and the anime model to get a mixture of them. That would be closer to the original image but the models might conflict and generate a hybrid.
3. With both models generating paired images, learn a pix2pix model.

## Extract the latent code of the face image
We need to attain the latent code corresponding to the given human image so that we can make use of it, and [rolux have done this](https://github.com/rolux/stylegan2encoder)!

> Note that we should replace *training/dataset.py* by *training/dataset-toonify.py*, and replace *dataset_tool.py* by *dataset_tool-toonify.py* here. The **-s2.py* files are backups.

In [ ]:
!python project_images.py --num-steps 500 images/aligned images/generated --network-pkl=pretrained_models/stylegan2-ffhq-512-config-f.pkl

## Generate Anime face with the latent code
We get the latent code of the given image now.

The simplest way to generate the anime face image is by inserting the code directly into the anime model.

> In this way, we assume the latent space between the human face model and the anime face model was the same.
>
> However, in fact, we just fine-tuned it so we could not guarantee such assumption would hold.
>
> So the output image would be a little different from the original image.

In [ ]:
!python generate_from_latent.py

## Blend the models
As [Justin Pinkney's work](https://colab.research.google.com/drive/1tputbmA9EaXs9HL9iO21g7xN7jz_Xrko?usp=sharing) shows, StyleGAN2 models can be blended easily.

We can get a blended model to generate an image between a human face and an anime face.

> Similarly we can transform a human face to a blended face.
>
> If you want to do so, you need to revise the file *generate_from_latent.py*.
>
> Replace *pretrained_models/ffhq-to-anime-512-config-f.pkl* by *pretrained_models/blended.pkl*.

In [ ]:
!python blend_models.py --low_res_pkl=pretrained_models/stylegan2-ffhq-512-config-f.pkl --high_res_pkl=pretrained_models/ffhq-to-anime-512-config-f.pkl --resolution=32 --output_pkl=pretrained_models/blended.pkl

## Train a pix2pix model
WIP

## Reference
[Analyzing and Improving the Image Quality of StyleGAN](https://arxiv.org/abs/1912.04958)

[Toonify yourself by Justin Pinkney](https://www.justinpinkney.com/toonify-yourself/)

[stylegan2encoder by rolux](https://github.com/rolux/stylegan2encoder)

[Making Anime Faces With StyleGAN](https://www.gwern.net/Faces)

[malnyun_faces by bryandlee](https://github.com/bryandlee/malnyun_faces)